# Classification

### Motivation

![Classification](./images/classification_motivation.png)

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# --- 1. DATA GENERATION (Replicating the Visualization's Dataset) ---

# Generate the concentric circles dataset
X, y = make_circles(n_samples=1000, noise=0.0, factor=0.5, random_state=42)

# Add the non-linear features used in the visualization to the input (Feature Engineering)
X_1_sq = X[:, 0:1] ** 2
X_2_sq = X[:, 1:2] ** 2
X_1_X_2 = (X[:, 0:1] * X[:, 1:2])
sin_X_1 = np.sin(X[:, 0:1])
sin_X_2 = np.sin(X[:, 1:2])

# Combine all features: [X1, X2, X1^2, X2^2, X1*X2, sin(X1), sin(X2)]
X_engineered = np.hstack((X, X_1_sq, X_2_sq, X_1_X_2, sin_X_1, sin_X_2))

# Split data (Visualization uses 50% test split, but we'll use 80/20 standard)
# If you want 50/50, change test_size to 0.5
X_train, X_test, y_train, y_test = train_test_split(
    X_engineered, y, test_size=0.2, random_state=42
)

# --- 2. MODEL ARCHITECTURE (Replicating the Visualization's Structure) ---

# Input: 7 features
# Hidden Layer 1: 4 neurons, Tanh
# Hidden Layer 2: 2 neurons, Tanh
# Output Layer: 1 neuron, Sigmoid (for binary classification probability)

# Initialize a sequential model
model = Sequential()

# Input layer (implicitly handled) and First Hidden Layer
model.add(Dense(
    units=4,                        # 4 neurons
    activation='tanh',              # Tanh activation
    input_shape=(X_engineered.shape[1],) # 7 input features
))

# Second Hidden Layer
model.add(Dense(
    units=2,                        # 2 neurons
    activation='tanh'               # Tanh activation
))

# Output Layer for Binary Classification
# Sigmoid outputs a probability (0 to 1) for the positive class
model.add(Dense(
    units=1,                        # 1 neuron
    activation='sigmoid'
))

# --- 3. COMPILATION and TRAINING ---

# Loss Function: Binary Cross-Entropy (standard for binary classification)
# Optimizer: Adam, with the specified Learning Rate
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.03), # Matching the Learning Rate: 0.03
    metrics=['accuracy']
)

# Train the model (for 100 epochs, similar to the 1360 in the image, to achieve high accuracy)
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=10, # Matching the Batch size: 10
    verbose=0,     # Suppress training output for brevity
    validation_data=(X_test, y_test)
)

# --- 4. EVALUATION (Checking the loss) ---

train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Display the results
print(f"Model trained with {X_engineered.shape[1]} engineered features.")
print(f"Training Accuracy: {train_acc:.4f} | Training Loss: {train_loss:.4f}")
print(f"Test Accuracy:     {test_acc:.4f} | Test Loss:     {test_loss:.4f}")

# The model should achieve 1.0000 accuracy and nearly 0.0000 loss,
# perfectly replicating the result in the screenshot, thanks to the
# engineered features.

/Users/sathnindu/Develop/SLIIT/year-4/dl/DL-Lab-01/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model trained with 7 engineered features.
Training Accuracy: 1.0000 | Training Loss: 0.0000
Test Accuracy:     1.0000 | Test Loss:     0.0000
